In [ ]:
# [1] Setup Environment & Clone Repo
import os
if os.path.exists('/kaggle/working'):
    os.chdir('/kaggle/working')
    !rm -rf ai_project

!git clone https://github.com/yangzeha/ai_project.git
%cd ai_project
!pip install -r TSB_CL_Project/requirements.txt
!apt-get update && apt-get install -y libsparsehash-dev


In [ ]:
# [2] Compile MSBE (C++)
!g++ -O3 Similar-Biclique-Idx-main/main.cpp -o Similar-Biclique-Idx-main/msbe
!chmod +x Similar-Biclique-Idx-main/msbe


In [ ]:
%%writefile TSB_CL_Project/prepare_yelp2018.py
# [3] Create Data Preparation Script
import os
import urllib.request
import shutil

def download_yelp2018():
    # 1. Setup Paths
    base_dir = os.path.dirname(os.path.abspath(__file__))
    dataset_dir = os.path.join(base_dir, "datasets")
    if not os.path.exists(dataset_dir):
        os.makedirs(dataset_dir)
        
    output_path = os.path.join(dataset_dir, "yelp2018.txt")
    
    # URLs from kuandeng/LightGCN repository
    base_url = "https://raw.githubusercontent.com/kuandeng/LightGCN/master/Data/yelp2018"
    files = ["train.txt", "test.txt"]
    
    print("Downloading Yelp2018 dataset...")
    
    all_interactions = []
    
    for filename in files:
        url = f"{base_url}/{filename}"
        print(f"Downloading {filename} from {url}...")
        try:
            with urllib.request.urlopen(url) as response:
                content = response.read().decode('utf-8')
                
                # Parse LightGCN format: UserID ItemID1 ItemID2 ...
                for line in content.strip().split('\n'):
                    parts = list(map(int, line.strip().split()))
                    if len(parts) < 2: continue
                    user = parts[0]
                    items = parts[1:]
                    for item in items:
                        all_interactions.append((user, item))
                        
        except Exception as e:
            print(f"Failed to download {filename}: {e}")
            return

    if not all_interactions:
        print("Error: No data downloaded.")
        return

    # Remap IDs to ensure continuity (0...N-1)
    print("Processing data...")
    users = sorted(list(set(x[0] for x in all_interactions)))
    items = sorted(list(set(x[1] for x in all_interactions)))
    
    user_map = {u: i for i, u in enumerate(users)}
    item_map = {i: i for i, i in enumerate(items)} # Items usually start from 0 in LightGCN data but let's be safe
    
    num_users = len(users)
    num_items = len(items)
    num_edges = len(all_interactions)
    
    print(f"Stats: Users={num_users}, Items={num_items}, Interactions={num_edges}")
    
    # Save to txt format for data_utils.py
    print(f"Saving to {output_path}...")
    with open(output_path, 'w') as f:
        # Header
        f.write(f"{num_users} {num_items} {num_edges}\n")
        # Data
        for u, i in all_interactions:
            f.write(f"{user_map[u]} {item_map[i]}\n")
            
    print("Done! Yelp2018 dataset is ready.")
    print(f"Path: {output_path}")

if __name__ == "__main__":
    download_yelp2018()


In [ ]:
# [4] Download & Process Yelp2018
!python TSB_CL_Project/prepare_yelp2018.py


In [ ]:
%%writefile TSB_CL_Project/quick_proof_yelp.py
# [5] Create Quick Proof Script
import os
os.environ["KMP_DUPLICATE_LIB_OK"] = "TRUE"

import torch
import torch.optim as optim
import numpy as np
import random
import time
from model import TSB_CL
from data_utils import DataUtils

# --- Configuration ---
LR = 0.001
BATCH_SIZE = 2048
EPOCHS = 20
EMBEDDING_DIM = 64
TAU = 2
EPSILON = 0.1

def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)

set_seed(2024)

def run_quick_proof():
    print("=== Running Quick Proof on Yelp2018 ===")
    
    # 1. Setup Paths
    CURRENT_DIR = os.path.dirname(os.path.abspath(__file__))
    PROJECT_ROOT = os.path.dirname(CURRENT_DIR)
    
    # Path to the newly prepared Yelp2018 dataset
    DATA_PATH = os.path.join(CURRENT_DIR, "datasets", "yelp2018.txt")
    
    # Path to MSBE executable
    exe_name = "msbe.exe" if os.name == 'nt' else "msbe"
    MSBE_EXE = os.path.join(PROJECT_ROOT, "Similar-Biclique-Idx-main", exe_name)

    if not os.path.exists(DATA_PATH):
        print(f"Error: Dataset not found at {DATA_PATH}")
        print("Please run 'prepare_yelp2018.py' first.")
        return

    if not os.path.exists(MSBE_EXE):
        print(f"Warning: MSBE executable not found at {MSBE_EXE}")
        print("Biclique mining might fail if not cached.")

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"Device: {device}")
    print(f"Data Path: {DATA_PATH}")

    # 2. Load Data
    utils = DataUtils(DATA_PATH, MSBE_EXE)
    all_data = utils.load_data()
    
    # 3. Split Data
    # OPTIMIZATION: Sample 10% of data to make mining faster for "Quick Proof"
    print("Sampling 10% of data for rapid verification...")
    random.shuffle(all_data)
    all_data = all_data[:int(len(all_data) * 0.1)]
    
    split_idx = int(len(all_data) * 0.8)
    train_data = all_data[:split_idx]
    test_data = all_data[split_idx:]
    
    print(f"Train size: {len(train_data)}, Test size: {len(test_data)}")
    
    # 4. Prepare Graph Structures
    print("Building Adjacency Matrix...")
    adj_matrix = utils.build_adj_matrix(train_data).to(device)
    
    print("Mining Bicliques from Training Data...")
    biclique_file = utils.run_msbe_mining(train_data, "yelp_train_sample", tau=TAU, epsilon=EPSILON)
    H_v, H_u = utils.parse_bicliques(biclique_file)
    H_v = H_v.to(device)
    H_u = H_u.to(device)
    
    print(f"Bicliques loaded. H_u shape: {H_u.shape}, H_v shape: {H_v.shape}")
    
    # 5. Define Models
    print("Initializing Models...")
    # TSB-CL Model (Static Version)
    model_tsb = TSB_CL(utils.num_users, utils.num_items, EMBEDDING_DIM).to(device)
    # Baseline (LightGCN equivalent)
    model_base = TSB_CL(utils.num_users, utils.num_items, EMBEDDING_DIM).to(device) 
    
    opt_tsb = optim.Adam(model_tsb.parameters(), lr=LR)
    opt_base = optim.Adam(model_base.parameters(), lr=LR)
    
    # 6. Evaluation Function
    test_users = list(set([x[0] for x in test_data]))
    if len(test_users) > 1000:
        eval_users = random.sample(test_users, 1000)
    else:
        eval_users = test_users
        
    test_user_ground_truth = {}
    for u, i, _ in test_data:
        if u not in test_user_ground_truth: test_user_ground_truth[u] = set()
        test_user_ground_truth[u].add(i)
        
    train_user_items = {}
    for u, i, _ in train_data:
        if u not in train_user_items: train_user_items[u] = set()
        train_user_items[u].add(i)

    def evaluate(model, use_biclique, model_name):
        model.eval()
        with torch.no_grad():
            if use_biclique:
                u_g, u_l, u_final, i_emb = model(adj_matrix, (H_v, H_u))
            else:
                dummy_Hv = torch.sparse_coo_tensor(size=(1, utils.num_items)).to(device)
                dummy_Hu = torch.sparse_coo_tensor(size=(utils.num_users, 1)).to(device)
                u_g, u_l, u_final, i_emb = model(adj_matrix, (dummy_Hv, dummy_Hu))
            
            # Use u_final for prediction
            u_emb = u_final
            
            hits = 0
            ndcgs = 0
            
            for u in eval_users:
                if u not in test_user_ground_truth: continue
                gt = test_user_ground_truth[u]
                
                scores = torch.matmul(u_emb[u], i_emb.t())
                
                # Mask training items
                if u in train_user_items:
                    mask_indices = list(train_user_items[u])
                    scores[mask_indices] = -float('inf')
                
                _, indices = torch.topk(scores, 20)
                preds = indices.cpu().numpy()
                
                # Recall
                hit = len(set(preds) & gt)
                hits += hit / len(gt)
                
                # NDCG
                dcg = 0
                idcg = 0
                for i, item in enumerate(preds):
                    if item in gt:
                        dcg += 1.0 / np.log2(i + 2)
                for i in range(min(len(gt), 20)):
                    idcg += 1.0 / np.log2(i + 2)
                ndcgs += dcg / idcg if idcg > 0 else 0
                
            return hits / len(eval_users), ndcgs / len(eval_users)

    # 7. Training Loop
    print(f"Starting Training for {EPOCHS} epochs...")
    
    users_np = np.array([x[0] for x in train_data])
    items_np = np.array([x[1] for x in train_data])
    num_batches = len(train_data) // BATCH_SIZE
    
    for epoch in range(EPOCHS):
        # Shuffle
        perm = np.random.permutation(len(train_data))
        users_np = users_np[perm]
        items_np = items_np[perm]
        
        model_tsb.train()
        model_base.train()
        
        total_loss_tsb = 0
        total_loss_base = 0
        
        for i in range(num_batches):
            start_idx = i * BATCH_SIZE
            end_idx = min((i + 1) * BATCH_SIZE, len(train_data))
            
            batch_users = torch.LongTensor(users_np[start_idx:end_idx]).to(device)
            batch_pos = torch.LongTensor(items_np[start_idx:end_idx]).to(device)
            batch_neg = torch.randint(0, utils.num_items, (len(batch_users),)).to(device)
            
            # --- Train TSB-CL ---
            u_g, u_l, u_final, i_final = model_tsb(adj_matrix, (H_v, H_u))
            
            u_emb = u_final[batch_users]
            pos_emb = i_final[batch_pos]
            neg_emb = i_final[batch_neg]
            
            pos_scores = torch.sum(u_emb * pos_emb, dim=1)
            neg_scores = torch.sum(u_emb * neg_emb, dim=1)
            
            loss_bpr = -torch.mean(torch.log(torch.sigmoid(pos_scores - neg_scores) + 1e-8))
            
            # Contrastive Loss
            u_g_norm = torch.nn.functional.normalize(u_g[batch_users], dim=1)
            u_l_norm = torch.nn.functional.normalize(u_l[batch_users], dim=1)
            pos_sim = torch.sum(u_g_norm * u_l_norm, dim=1)
            loss_cl = -torch.mean(torch.log(torch.sigmoid(pos_sim / 0.2) + 1e-8))
            
            loss_tsb = loss_bpr + 0.1 * loss_cl
            
            opt_tsb.zero_grad()
            loss_tsb.backward()
            opt_tsb.step()
            total_loss_tsb += loss_tsb.item()
            
            # --- Train Baseline ---
            dummy_Hv = torch.sparse_coo_tensor(size=(1, utils.num_items)).to(device)
            dummy_Hu = torch.sparse_coo_tensor(size=(utils.num_users, 1)).to(device)
            u_g_b, _, u_final_b, i_final_b = model_base(adj_matrix, (dummy_Hv, dummy_Hu))
            
            u_emb_b = u_final_b[batch_users]
            pos_emb_b = i_final_b[batch_pos]
            neg_emb_b = i_final_b[batch_neg]
            
            pos_scores_b = torch.sum(u_emb_b * pos_emb_b, dim=1)
            neg_scores_b = torch.sum(u_emb_b * neg_emb_b, dim=1)
            
            loss_base = -torch.mean(torch.log(torch.sigmoid(pos_scores_b - neg_scores_b) + 1e-8))
            
            opt_base.zero_grad()
            loss_base.backward()
            opt_base.step()
            total_loss_base += loss_base.item()
            
        if (epoch + 1) % 5 == 0:
            print(f"Epoch {epoch+1}/{EPOCHS} | Loss TSB: {total_loss_tsb:.4f} | Loss Base: {total_loss_base:.4f}")

    print("\n=== Final Evaluation (Recall@20) ===")
    recall_tsb, ndcg_tsb = evaluate(model_tsb, True, "TSB-CL")
    recall_base, ndcg_base = evaluate(model_base, False, "Baseline")
    
    print(f"TSB-CL   -> Recall: {recall_tsb:.4f} | NDCG: {ndcg_tsb:.4f}")
    print(f"Baseline -> Recall: {recall_base:.4f} | NDCG: {ndcg_base:.4f}")
    
    if recall_tsb > recall_base:
        print(f"\nSUCCESS: TSB-CL outperforms Baseline by +{(recall_tsb - recall_base)*100:.2f}%")
    else:
        print("\nResult is close. Try tuning tau/epsilon for sparse data.")

if __name__ == "__main__":
    run_quick_proof()


In [ ]:
# [6] Run Quick Proof (Static TSB-CL vs LightGCN)
!python TSB_CL_Project/quick_proof_yelp.py
